In [1]:
import pandas as pd

In [2]:
# Paths
election_results_path = '../dataFiles/processedData/countypres_2000-2016_processed.csv'
religion_path = '../dataFiles/processedData/religion_by_county_2010_processed.csv'
demographics_path = '../dataFiles/processedData/FinalDemographics_cleaned.xlsx'
education_path = '../dataFiles/processedData/education_processed.xlsx'

# Get Data
election_results = pd.read_csv(election_results_path)
religion = pd.read_csv(religion_path)
demographics = pd.read_excel(demographics_path, sheet_name='Sheet1')
education = pd.read_excel(education_path, sheet_name='Sheet1')

In [3]:
# Make Election Results FIPS 5 Digits
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
election_results['FIPS'] = election_results['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
election_results.insert(len(election_results.columns), 'Add Zero FIPS', '0' + election_results['FIPS'])
election_results.loc[election_results['FIPS'].str.len()==4, 'FIPS'] = election_results.loc[election_results['FIPS'].str.len()==4, 'Add Zero FIPS']
del election_results['Add Zero FIPS']

election_results['totalvotes'] = election_results['totalvotes']
election_results.insert(len(election_results.columns), 'dem_votes', election_results['totalvotes']*election_results['dem_vote_share'])
election_results.insert(len(election_results.columns), 'rep_votes', election_results['totalvotes']*election_results['rep_vote_share'])
election_results.insert(len(election_results.columns), 'other_votes', election_results['totalvotes']*election_results['other_vote_share'])
# Change %s To Doubles
election_results['dem_vote_share'] = election_results['dem_vote_share']*100
election_results['rep_vote_share'] = election_results['rep_vote_share']*100
election_results['other_vote_share'] = election_results['other_vote_share']*100
election_results['republican_minus_democratic'] = election_results['republican_minus_democratic']*100
election_results.head()

,FIPS,state,county,year,FIPS/Year,totalvotes,dem_vote_share,rep_vote_share,other_vote_share,winning_party,republican_minus_democratic,dem_votes,rep_votes,other_votes
0,01001,Alabama,Autauga,2008,2008/01001,23641,25.773021,73.613637,0.613341,Republican,47.840616,6093.0,17403.0,145.0
1,01003,Alabama,Baldwin,2008,2008/01003,81413,23.811922,75.259479,0.928599,Republican,51.447558,19386.0,61271.0,756.0
2,01005,Alabama,Barbour,2008,2008/01005,11630,48.985383,50.438521,0.576096,Republican,1.453138,5697.0,5866.0,67.0
3,01007,Alabama,Bibb,2008,2008/01007,8644,26.596483,72.443313,0.960204,Republican,45.846830,2299.0,6262.0,83.0
4,01009,Alabama,Blount,2008,2008/01009,24267,14.513537,84.019450,1.467013,Republican,69.505913,3522.0,20389.0,356.0


In [4]:
# Merge Religion
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
religion['FIPS'] = religion['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
religion.insert(len(religion.columns), 'Add Zero FIPS', '0' + religion['FIPS'])
religion.loc[religion['FIPS'].str.len()==4, 'FIPS'] = religion.loc[religion['FIPS'].str.len()==4, 'Add Zero FIPS']
del religion['Add Zero FIPS']
# Make Religion Same For 2008, 2012, 2016
religion_2008 = religion.copy()
religion_2012 = religion.copy()
religion_2016 = religion.copy()
religion_2008['Year'] = '2008'
religion_2012['Year'] = '2012'
religion_2016['Year'] = '2016'
religion = religion_2008.append(religion_2012).append(religion_2016).reset_index(drop=True)
# Make FIPS/Year Lookup Key
religion.insert(len(religion.columns), 'FIPS/Year',religion['Year'].astype(str) + '/' + religion['FIPS'])
# Merge Religion Using Lookup Key
religion.rename(columns={'Other':'Other Religion'}, inplace=True)
election_results = election_results.merge(religion[['FIPS/Year','Evangelical Protestant','Black Protestant', 'Mainline Protestant','Catholic','Orthodox','Other Religion']], how='left', on='FIPS/Year')
# Change %s to Doubles
election_results['Evangelical Protestant'] = election_results['Evangelical Protestant']*100
election_results['Black Protestant'] = election_results['Black Protestant']*100
election_results['Mainline Protestant'] = election_results['Mainline Protestant']*100
election_results['Catholic'] = election_results['Catholic']*100
election_results['Orthodox'] = election_results['Orthodox']*100
election_results['Other Religion'] = election_results['Other Religion']*100
election_results.head()

,FIPS,state,county,year,FIPS/Year,totalvotes,dem_vote_share,rep_vote_share,other_vote_share,winning_party,republican_minus_democratic,dem_votes,rep_votes,other_votes,Evangelical Protestant,Black Protestant,Mainline Protestant,Catholic,Orthodox,Other Religion
0,01001,Alabama,Autauga,2008,2008/01001,23641,25.773021,73.613637,0.613341,Republican,47.840616,6093.0,17403.0,145.0,50.399000,4.197889,8.285889,3.235889,NaN,1.568889
1,01003,Alabama,Baldwin,2008,2008/01003,81413,23.811922,75.259479,0.928599,Republican,51.447558,19386.0,61271.0,756.0,31.813889,1.717000,11.014000,7.685889,0.104,0.838000
2,01005,Alabama,Barbour,2008,2008/01005,11630,48.985383,50.438521,0.576096,Republican,1.453138,5697.0,5866.0,67.0,32.025000,12.120889,7.793889,2.094000,NaN,0.965000
3,01007,Alabama,Bibb,2008,2008/01007,8644,26.596483,72.443313,0.960204,Republican,45.846830,2299.0,6262.0,83.0,44.332889,4.215889,1.317889,NaN,NaN,0.013000
4,01009,Alabama,Blount,2008,2008/01009,24267,14.513537,84.019450,1.467013,Republican,69.505913,3522.0,20389.0,356.0,50.980000,0.101000,5.295000,8.276000,NaN,0.508889


In [5]:
# Merge Demographics
del demographics['FIPS/Year']
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
demographics['FIPS'] = demographics['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
demographics.insert(len(religion.columns), 'Add Zero FIPS', '0' + demographics['FIPS'])
demographics.loc[demographics['FIPS'].str.len()==4, 'FIPS'] = demographics.loc[demographics['FIPS'].str.len()==4, 'Add Zero FIPS']
del demographics['Add Zero FIPS']
# Make FIPS/Year Lookup Key
demographics.insert(len(demographics.columns), 'FIPS/Year',demographics['year'].astype(str) + '/' + demographics['FIPS'])
demographics.insert(len(demographics.columns), '% Single Parent Households',demographics['% Households: Family Households: Other Family: Male Householder, No Wife Present'] + demographics['% Households: Family Households: Other Family: Female Householder, No Husband Present'])

#Get Columns
columns_to_get = ['FIPS/Year',
'Total Population',
'Population Density (Per Sq. Mile)',
'% Total Population: Male', 
'% Total Population: Female',
'% Total Population: Under 18 Years',
'% Total Population: 18 to 34 Years',
'% Total Population: 35 to 64 Years',
'% Total Population: 65 and Over',
'% Total Population: White Alone',
'% Total Population: Black or African American Alone',
'% Total Population: American Indian and Alaska Native Alone',
'% Total Population: Asian Alone',
'% Total Population: Native Hawaiian and Other Pacific Islander Alone',
'% Total Population: Some Other Race Alone',
'% Total Population: Two or More Races',
'% Population 15 Years and Over: Never Married',
'% Population 15 Years and Over: Now Married (Not Including Separated)',
'% Population 15 Years and Over: Separated',
'% Population 15 Years and Over: Widowed',
'% Population 15 Years and Over: Divorced',
'Average Household Size',
'Gini Index',
'% Civilian Population 18 Years and Over: Veteran',
'% Civilian Population 18 Years and Over: Nonveteran',
'% Single Parent Households']

# Merge Demographics Using Lookup Key
election_results = election_results.merge(demographics[columns_to_get], how='left', on='FIPS/Year')
election_results.head()

,FIPS,state,county,year,FIPS/Year,totalvotes,dem_vote_share,rep_vote_share,other_vote_share,winning_party,...,% Population 15 Years and Over: Never Married,% Population 15 Years and Over: Now Married (Not Including Separated),% Population 15 Years and Over: Separated,% Population 15 Years and Over: Widowed,% Population 15 Years and Over: Divorced,Average Household Size,Gini Index,% Civilian Population 18 Years and Over: Veteran,% Civilian Population 18 Years and Over: Nonveteran,% Single Parent Households
0,01001,Alabama,Autauga,2008,2008/01001,23641,25.773021,73.613637,0.613341,Republican,...,26.04,52.20,2.50,6.65,12.61,2.72,0.4122,15.39,84.61,17.74
1,01003,Alabama,Baldwin,2008,2008/01003,81413,23.811922,75.259479,0.928599,Republican,...,21.50,58.21,1.51,6.85,11.93,2.50,0.4080,15.54,84.46,14.37
2,01005,Alabama,Barbour,2008,2008/01005,11630,48.985383,50.438521,0.576096,Republican,...,33.05,42.73,3.10,7.61,13.50,2.60,0.4758,10.13,89.87,22.72
3,01007,Alabama,Bibb,2008,2008/01007,8644,26.596483,72.443313,0.960204,Republican,...,26.82,47.53,3.82,7.49,14.34,2.95,0.4212,9.44,90.56,21.66
4,01009,Alabama,Blount,2008,2008/01009,24267,14.513537,84.019450,1.467013,Republican,...,20.13,57.99,2.09,7.81,11.98,2.71,0.4112,9.77,90.23,14.68


In [6]:
# Clean Education
# Drop Duplicates
education = education.drop_duplicates(subset=['FIPS','Year']).reset_index(drop=True)
# Only Keep 2012 and 2016
education_2012 = education.loc[education['Year']==2012,:].copy().reset_index(drop=True)
education_2016 = education.loc[education['Year']==2016,:].copy().reset_index(drop=True)
# Make 2008 the Same As 2012
education_2008 = education_2012.copy()
education_2008['Year'] = 2008
education = education_2008.append(education_2012).append(education_2016)
# Convert To Percentages
education["High school graduate (includes equivalency)"] = education["High school graduate (includes equivalency)"]/education["Total"]*100
education["Some college or associate's degree"] = education["Some college or associate's degree"]/education["Total"]*100
education["Bachelor's degree"] = education["Bachelor's degree"]/education["Total"]*100
education["Graduate or professional degree"] = education["Graduate or professional degree"]/education["Total"]*100
education.insert(len(education.columns), "College or Above", education["Bachelor's degree"] + education["Graduate or professional degree"])
education = education[['FIPS', 'Year', 'College or Above']].copy()

In [7]:
# Merge Education
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
education['FIPS'] = education['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
education.insert(len(education.columns), 'Add Zero FIPS', '0' + education['FIPS'])
education.loc[education['FIPS'].str.len()==4, 'FIPS'] = education.loc[education['FIPS'].str.len()==4, 'Add Zero FIPS']
del education['Add Zero FIPS']
# Make FIPS/Year Lookup Key
education.insert(len(education.columns), 'FIPS/Year',education['Year'].astype(str) + '/' + education['FIPS'])
# Merge Education Using Lookup Key
election_results = election_results.merge(education[['FIPS/Year','College or Above']], how='left', on='FIPS/Year')
election_results.head()

,FIPS,state,county,year,FIPS/Year,totalvotes,dem_vote_share,rep_vote_share,other_vote_share,winning_party,...,% Population 15 Years and Over: Now Married (Not Including Separated),% Population 15 Years and Over: Separated,% Population 15 Years and Over: Widowed,% Population 15 Years and Over: Divorced,Average Household Size,Gini Index,% Civilian Population 18 Years and Over: Veteran,% Civilian Population 18 Years and Over: Nonveteran,% Single Parent Households,College or Above
0,01001,Alabama,Autauga,2008,2008/01001,23641,25.773021,73.613637,0.613341,Republican,...,52.20,2.50,6.65,12.61,2.72,0.4122,15.39,84.61,17.74,21.707831
1,01003,Alabama,Baldwin,2008,2008/01003,81413,23.811922,75.259479,0.928599,Republican,...,58.21,1.51,6.85,11.93,2.50,0.4080,15.54,84.46,14.37,29.144964
2,01005,Alabama,Barbour,2008,2008/01005,11630,48.985383,50.438521,0.576096,Republican,...,42.73,3.10,7.61,13.50,2.60,0.4758,10.13,89.87,22.72,14.524286
3,01007,Alabama,Bibb,2008,2008/01007,8644,26.596483,72.443313,0.960204,Republican,...,47.53,3.82,7.49,14.34,2.95,0.4212,9.44,90.56,21.66,8.996005
4,01009,Alabama,Blount,2008,2008/01009,24267,14.513537,84.019450,1.467013,Republican,...,57.99,2.09,7.81,11.98,2.71,0.4112,9.77,90.23,14.68,12.381469


In [8]:
# Economic Data


In [9]:
election_results

,FIPS,state,county,year,FIPS/Year,totalvotes,dem_vote_share,rep_vote_share,other_vote_share,winning_party,...,% Population 15 Years and Over: Now Married (Not Including Separated),% Population 15 Years and Over: Separated,% Population 15 Years and Over: Widowed,% Population 15 Years and Over: Divorced,Average Household Size,Gini Index,% Civilian Population 18 Years and Over: Veteran,% Civilian Population 18 Years and Over: Nonveteran,% Single Parent Households,College or Above
0,01001,Alabama,Autauga,2008,2008/01001,23641,25.773021,73.613637,0.613341,Republican,...,52.20,2.50,6.65,12.61,2.72,0.4122,15.39,84.61,17.74,21.707831
1,01003,Alabama,Baldwin,2008,2008/01003,81413,23.811922,75.259479,0.928599,Republican,...,58.21,1.51,6.85,11.93,2.50,0.4080,15.54,84.46,14.37,29.144964
2,01005,Alabama,Barbour,2008,2008/01005,11630,48.985383,50.438521,0.576096,Republican,...,42.73,3.10,7.61,13.50,2.60,0.4758,10.13,89.87,22.72,14.524286
3,01007,Alabama,Bibb,2008,2008/01007,8644,26.596483,72.443313,0.960204,Republican,...,47.53,3.82,7.49,14.34,2.95,0.4212,9.44,90.56,21.66,8.996005
4,01009,Alabama,Blount,2008,2008/01009,24267,14.513537,84.019450,1.467013,Republican,...,57.99,2.09,7.81,11.98,2.71,0.4112,9.77,90.23,14.68,12.381469
5,01011,Alabama,Bullock,2008,2008/01011,5415,74.072022,25.687904,0.240074,Democratic,...,32.79,6.44,9.73,11.10,2.75,0.4979,8.80,91.20,28.99,11.881188
6,01013,Alabama,Butler,2008,2008/01013,9709,43.135235,56.493975,0.370790,Republican,...,42.67,3.86,10.14,14.12,2.51,0.4691,10.73,89.27,25.90,12.868404
7,01015,Alabama,Calhoun,2008,2008/01015,49242,33.170870,65.691889,1.137241,Republican,...,50.81,3.60,8.28,12.30,2.37,0.4280,13.08,86.92,18.31,16.916291
8,01017,Alabama,Chambers,2008,2008/01017,14956,45.460016,53.938219,0.601765,Republican,...,45.16,3.93,9.61,13.23,2.48,0.4576,10.36,89.64,22.79,11.024089
9,01019,Alabama,Cherokee,2008,2008/01019,9745,23.663417,74.889687,1.446896,Republican,...,67.07,1.60,7.54,10.30,2.17,0.4556,9.99,90.01,12.98,13.127601


In [10]:
# Export Final Dataframe
output_path  = '../dataFiles/mergedData/election_results_with_variables.xlsx'
election_results.to_excel(output_path,index=False)